# Load GEMD files in database
This code is for information purposes onlny as it requires permissions on our SciServer system that users of this repo will not in general posess. The database schema is defined by files in the database_design subfolder.

#### NB parts of this code can only be run by someone with direct credentials to the database

In [ ]:
import glob
import json
import random
import pandas
import pprint
import sys
import json
import numpy as np

## create CSV files from the JSON GEMD 
complete_data_model.json is located in an aptly named folder and contains all JSON content to be loaded

In [ ]:
maggie='/home/idies/workspace/Temporary/gerard/GEMD-Maggie'

In [ ]:
with open(f'{maggie}/gemd_data_model_dumps/complete_data_model.json','r') as lf:
    d=json.load(lf)

In [ ]:
len(d['context']),len(d['object'])

In [ ]:
contexts=[[c['uids']['auto'],c['type'],json.dumps(c)] for c in d['context']]
contexts=pandas.DataFrame(contexts,columns=['uid','gemd_type','context'])

In [ ]:
objects=[[o['id'],o['scope'],o['type']] for o in d['object']]
objects=pandas.DataFrame(objects,columns=['uid','scope','gemd_type'])

In [ ]:
CONTEXTS_CSV=f'{maggie}/contexts.csv'
OBJECTS_CSV=f'{maggie}/objects.csv'

In [ ]:
contexts.to_csv(CONTEXTS_CSV,index=False,header=False)

In [ ]:
objects=objects.drop_duplicates()   # there were some
objects.to_csv(OBJECTS_CSV,index=False,header=False)

# Load in GEMD database
Prepared with 
```sql
create table GEMDContext (
  uid varchar(36) not null
  , gemd_type varchar(32) not null
  , context varchar(max) not null
)

create table GEMDObject (
  uid varchar(36) not null
  , scope varchar(32) not null
  , gemd_type varchar(32) not null
)
```

Here starts some code that is only runnable by GL

In [ ]:
pyUTILS_path="/path/to/a/python/script/with/database/access/utility/functions"
AUTH_JSON='location/of/file/with/database/credentials'

In [ ]:
sys.path.insert(0, pyUTILS_path)
from sciserver_db_geography_utils import *
with open(AUTH_JSON,"r") as f:
    AUTH=json.load(f)

Next call submits a Job to SciServer batch queue that uses Java code to load data into the database. More efficient than for example SQL Alchemy uploads.

#### NB this code can only be run by someone with direct credentials to the database

# add more explicit tables and views
For each of the main object types create table/view and extract somewhat more explicit metadata into columns. In particular a name and all the references/foreign keys to other tables n this set. Note that the tables are defined implicitly using the <tt>SELECT ... INTO...</tt> supported by MS SQL. For this reason one needs a database user with appopriate privileges.

<img src='https://citrineinformatics.github.io/gemd-docs/img/GEMD-Object-Graph.png'></img>


Use class in utility script. Again must be laoded with appropriate permissions not avilable to random users.

In [ ]:
from gemd_database import MSSQLDatabase

In [ ]:
GEMDDB=MSSQLDatabase(AUTH,'GEMD')

## material

In [ ]:
sql="""
select uid
,      json_value(context,'$.name') as name
,      context
  into material_template
  from gemdcontext
 where gemd_type='material_template'
"""
r=GEMDDB.execute_update(sql)    

In [ ]:
sql="""
drop table material_spec
select uid
,      json_value(context,'$.name') as name
,      json_value(context,'$.template.id') as template_uid
,      json_value(context,'$.process.id') as process_spec_uid
,      context
  into material_spec
  from gemdcontext
 where gemd_type='material_spec'
"""
GEMDDB.execute_update(sql)

In [ ]:
sql="""
select uid
,      json_value(context,'$.name') as name
,      json_value(context,'$.spec.id') as spec_uid
,      json_value(context,'$.process.id') as process_run_uid
,      context
  into material_run
  from gemdcontext
 where gemd_type='material_run'
"""
GEMDDB.execute_update(sql)

## measurement

In [ ]:
sql="""
select uid
,      json_value(context,'$.name') as name
,      context
  into measurement_template
  from gemdcontext
 where gemd_type='measurement_template'
"""
GEMDDB.execute_update(sql)

In [ ]:
sql="""
select uid
,      json_value(context,'$.name') as name
,      json_value(context,'$.template.id') as template_uid
,      context
  into measurement_spec
  from gemdcontext
 where gemd_type='measurement_spec'
"""
GEMDDB.execute_update(sql)

In [ ]:
sql="""
select uid
,      json_value(context,'$.name') as name
,      json_value(context,'$.source.performed_by') as performed_by
,      json_value(context,'$.source.performed_date') as performed_date
,      json_value(context,'$.spec.id') as spec_uid
,      json_value(context,'$.material.id') as material_uid
,      context
  into measurement_run
  from gemdcontext
 where gemd_type='measurement_run'
"""
GEMDDB.execute_update(sql)

## process

In [ ]:
sql="""
select uid
,      json_value(context,'$.name') as name
,      context
  into process_template
  from gemdcontext
 where gemd_type='process_template'
"""
GEMDDB.execute_update(sql)

In [ ]:
sql="""
select uid
,      json_value(context,'$.name') as name
,      json_value(context,'$.source.performed_by') as p_performed_by
,      json_value(context,'$.source.performed_date') as p_performed_date
,      json_value(context,'$.spec.id') as spec_uid
,      context
  into process_run
  from gemdcontext
 where gemd_type='process_run'
"""
GEMDDB.execute_update(sql)

In [ ]:
sql="""
select uid
,      json_value(context,'$.name') as name
,      json_value(context,'$.template.id') as template_uid
,      context
  into process_spec
  from gemdcontext
 where gemd_type='process_spec'
"""
GEMDDB.execute_update(sql)

## ingredient

In [ ]:
# into ingredient_spec
#   
sql="""
select distinct s.gemd_type as spec, m.gemd_type as material, p.gemd_type as process
  from gemdcontext i 
  join gemdcontext s on s.uid=json_value(i.context,'$.spec.id')
  join gemdcontext m on m.uid=json_value(i.context,'$.material.id')
  join gemdcontext p on p.uid=json_value(i.context,'$.process.id')
 where i.gemd_type='ingredient_run'
"""
GEMDDB.execute_query(sql)
# GEMDDB.execute_update(sql)

In [ ]:
# 
#   
sql="""
select i.uid
,      json_value(i.context,'$.name') as name
,      json_value(i.context,'$.material.id') as material_spec_uid
,      json_value(i.context,'$.process.id') as process_spec_uid
,      i.context
  into ingredient_spec
  from gemdcontext i 
where i.gemd_type='ingredient_spec'
"""
# GEMDDB.execute_query(sql)
GEMDDB.execute_update(sql)

In [ ]:
sql="""
select i.uid
,      json_value(i.context,'$.name') as name
,      json_value(i.context,'$.spec.id') as spec_uid
,      json_value(i.context,'$.material.id') as material_run_uid
,      json_value(i.context,'$.process.id') as process_run_uid
,      i.context
  into ingredient_run
  from gemdcontext i 
where i.gemd_type='ingredient_run'
"""
# GEMDDB.execute_query(sql)
GEMDDB.execute_update(sql)